#### Paso 0

Lectura de los datasets:

In [1]:
# Importar las librerias necesarias
import pandas as pd

# Leer los datasets
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col = False, compression = "zip")

#### Paso 1

Realizar la limpieza de los datasets:

In [2]:
# Renombrar la columna "default payment next month" a "default".
test_data = test_data.rename(columns = {'default payment next month': 'default'})
train_data = train_data.rename(columns = {'default payment next month': 'default'})

# Remover la columna "ID".
test_data = test_data.drop(columns = ['ID'])
train_data = train_data.drop(columns = ['ID'])

# Remover las filas con valores faltantes en el dataset de entrenamiento
train_data = train_data.loc[train_data["MARRIAGE"] != 0]
train_data = train_data.loc[train_data["EDUCATION"] != 0]

# Remover las filas con valores faltantes en el dataset de prueba
test_data = test_data.loc[test_data["MARRIAGE"] != 0]
test_data = test_data.loc[test_data["EDUCATION"] != 0]

# Modificar la columna 'EDUCATION' para valores > 4, agrupar en "others"
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

#### Paso 2

Dividir los datasets en x_train, y_train, x_test, y_tes:

In [3]:
# Separar en (X) variables y variable objetivo (y)

# Conjunto de entrenamiento
X_train = train_data.drop(columns = ['default'])
y_train = train_data['default']

# Conjunto de prueba
X_test = test_data.drop(columns = ['default'])
y_test = test_data['default']

#### Paso 3

Crear un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:

##### Transformar las variables categoricas usando el método one-hot-encoding:

In [4]:
# Importar las librerias necesarias
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Seleccionar las columnas categóricas y numéricas
categorical_columns = ['SEX', 'EDUCATION', 'MARRIAGE']
numerical_columns = num_columns = [col for col in X_train.columns if col not in categorical_columns]

# Definir el transformador para las variables categóricas (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers = [
        ('cat', OneHotEncoder(), categorical_columns), 
        ('scaler', StandardScaler(), numerical_columns)
    ]
)

#### - Descompone la matriz de entrada usando componentes principales.
#### - El pca usa todas las componentes.
#### - Escala la matriz de entrada al intervalo [0, 1].
#### - Selecciona las K columnas mas relevantes de la matrix de entrada.
#### - Ajusta una red neuronal tipo MLP.

In [5]:
# Importar las librerias necesarias
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

# Crear el pipeline con preprocesamiento, PCA, selección de características y clasificador
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ('pca', PCA()),
    ('feature_selection', SelectKBest(score_func = f_classif)),
    ('classifier', MLPClassifier(max_iter = 15000, random_state = 21))
])

#### Paso 4

Optimizar los hiperparametros del pipeline usando validación cruzada. Usar 10 splits para la validación cruzada. Usar la función de precisión balanceada para medir la precisión del modelo:

In [6]:
# Importar las librerias necesarias
from sklearn.model_selection import GridSearchCV

# Definir los parámetros que se van a probar en el GridSearch
param_grid = {
    "pca__n_components": [None],
    "feature_selection__k": [20],
    "classifier__hidden_layer_sizes": [(50, 30, 40, 60)],
    "classifier__alpha": [0.26],
    'classifier__learning_rate_init': [0.001]
}

# Crear el modelo con GridSearch
model = GridSearchCV(
    pipeline,                      # El pipeline que definimos previamente
    param_grid,                    # Los hiperparámetros a probar
    cv = 10,                       # Número de splits para validación cruzada
    scoring = "balanced_accuracy", # Usamos precisión balanceada
    n_jobs = -1,                   # Usar todos los núcleos disponibles
    refit = True                   # Refinar el modelo con el mejor resultado
)

# Ajustar el modelo con el conjunto de entrenamiento
model.fit(X_train, y_train)

# Ver los mejores parámetros encontrados
print(f"Mejores hiperparámetros: {model.best_params_}")

Mejores hiperparámetros: {'classifier__alpha': 0.26, 'classifier__hidden_layer_sizes': (50, 30, 40, 60), 'classifier__learning_rate_init': 0.001, 'feature_selection__k': 20, 'pca__n_components': None}


##### Paso 5

Guardar el modelo como "files/models/model.pkl.gz":

In [7]:
# Importar las librerías necesarias
import os
import gzip
import pickle

# Crear carpeta si no existe
models_dir = '../files/models'
os.makedirs(models_dir, exist_ok = True)

# Definir la ruta donde se guardará el modelo
model_path = "../files/models/model.pkl.gz"

# Guardar el modelo entrenado en un archivo
with gzip.open(model_path, "wb") as model_file:
    pickle.dump(model, model_file)

print(f"Modelo guardado en '{model_path}'")

Modelo guardado en '../files/models/model.pkl.gz'


##### Paso 6

Calcular las metricas de precisión, precisión balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba. Guardarlas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. 

Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:

{'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}

{'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

In [8]:
# Importar las librerías necesarias
import json
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

# Hacer predicciones
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calcular métricas para el conjunto de entrenamiento
metrics_train = {
    'type': 'metrics',
    'dataset': 'train',
    'precision': precision_score(y_train, y_train_pred, zero_division=0),
    'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
    'recall': recall_score(y_train, y_train_pred, zero_division=0),
    'f1_score': f1_score(y_train, y_train_pred, zero_division=0)
}

# Calcular métricas para el conjunto de prueba
metrics_test = {
    'type': 'metrics',
    'dataset': 'test',
    'precision': precision_score(y_test, y_test_pred, zero_division=0),
    'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
    'recall': recall_score(y_test, y_test_pred, zero_division=0),
    'f1_score': f1_score(y_test, y_test_pred, zero_division=0)
 }

# Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
output_path = os.path.join(output_dir, 'metrics.json')
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
    f.write(json.dumps(metrics_train) + '\n')
    f.write(json.dumps(metrics_test) + '\n')

#### Paso 7

Calcular las matrices de confusión para los conjuntos de entrenamiento y prueba. Guardarlas en el archivo files/output/metrics.json. 

Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba. Por ejemplo:

{'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}

{'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}

In [9]:
# Importar las librerías necesarias
from sklearn.metrics import confusion_matrix

# Calcular matrices de confusión
cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

# Convertir las matrices de confusión en formato JSON
def format_confusion_matrix(cm, dataset_type):
    return {
        'type': 'cm_matrix',
        'dataset': dataset_type,
        'true_0': {
            'predicted_0': int(cm[0, 0]),
            'predicted_1': int(cm[0, 1])
        },
        'true_1': {
            'predicted_0': int(cm[1, 0]),
            'predicted_1': int(cm[1, 1])
        }
    }

metrics = [
    format_confusion_matrix(cm_train, 'train'),
    format_confusion_matrix(cm_test, 'test')
]

# Guardar las matrices de confusión en el mismo archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'a') as f:  # Usar 'a' para agregar después de las métricas
    for metric in metrics:
        f.write(json.dumps(metric) + '\n')